In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
from datetime import datetime
from sklearn.preprocessing import OneHotEncoder
from fonctions_preprocess import *

In [2]:
df = pd.read_csv("SOCCER_201223_18h.csv")
mapping_equipe2 = {
    'Nimes': 'Nîmes',
    'Paris S-G': 'Paris Saint Germain',
    'Saint Etienne': 'Saint-Étienne'
}

In [3]:
df = preprocess(df, mapping_equipe2)

In [4]:
df = preparation_model(df)

/home/onyxia/work/fonctions_preprocess.py:143: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Moyenne_{col}_Lag'] = df.groupby(['Saison', 'equipe'])[col].transform(lambda x: x.shift(1).expanding().mean())
/home/onyxia/work/fonctions_preprocess.py:143: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f'Moyenne_{col}_Lag'] = df.groupby(['Saison', 'equipe'])[col].transform(lambda x: x.shift(1).expanding().mean())
/home/onyxia/work/fonctions_preprocess.py:143: PerformanceWarning: DataFrame is highly fragmented.  This is usually 

In [5]:
pd.set_option('display.max_columns', None)



In [10]:
test_df = df.copy()
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

# Charger votre DataFrame TEST
# Assurez-vous que les variables "Venue" et "Opponent" sont des objets (str) pour l'encodage en variables factices
# Par exemple, si votre DataFrame est appelé test_df :
# test_df['Venue'] = test_df['Venue'].astype(str)
# test_df['Opponent'] = test_df['Opponent'].astype(str)

# Convertissez la colonne "DateTime" en format de date
# Convertissez la colonne "DateTime" en format de date
test_df['DateTime'] = pd.to_datetime(test_df['DateTime'])

variables_moyenne = df.filter(like='Moyenne_', axis=1)
variables_moyenne = variables_moyenne.columns.tolist()


test_df = test_df[["DateTime", "Comp", "Saison", "Round", "Formation", "Venue", "equipe","Result",	"GF",	"GA",	"Opponent", "GD", "Points", "Points_Cum", "GD_Cum", "Classement", "GF_Cum", "GA_Cum","CumulativeLosses","Past_Matches", "CumulativeWins", "CumulativeDraws", "Points_Cum_Lag1", "GD_Cum_Lag1", "GF_Cum_Lag1", "GA_Cum_Lag1", "Formation"]+variables_moyenne]
# Supprimez les lignes avec des valeurs NaN dans l'ensemble de la base de données
test_df.dropna(inplace=True)

# Filtrer les matchs avec "Round" > 5
test_df = test_df[test_df['Round'] > 0]

# Fixez la date de coupure (cutoff) en juillet 2023
cutoff_date = pd.to_datetime('2021-12-12')


# Sélectionner les variables numériques
num_features = ["Past_Matches", "Moyenne_Standard_SoT%"] +variables_moyenne
#"CPoints_Lag1", "CGD_Lag1", "Classement_Lag1"
#, "CumulativeWins", "CumulativeDraws","CumulativeLosses",

#"Moyenne_Total_Cmp%", "Moyenne_Poss_x",'Moyenne_Touches_Def_Pen',  'Moyenne_Touches_Def_3rd']

# Sélectionner les variables catégorielles
cat_features = ["Venue", "Opponent", "Formation"]


# Utilisez pd.get_dummies pour encoder les variables catégorielles en variables factices
encoded_cat_features = pd.get_dummies(test_df[cat_features])

# Sélectionnez les variables d'entraînement et de test en combinant les numériques et les factices
X = pd.concat([test_df[num_features], encoded_cat_features], axis=1)
y = test_df['Result']

# Séparez les données en ensembles d'entraînement et de test en utilisant la date de coupure
X_train = X[test_df['DateTime'] <= cutoff_date]
y_train = y[test_df['DateTime'] <= cutoff_date]
X_test = X[test_df['DateTime'] > cutoff_date]
y_test = y[test_df['DateTime'] > cutoff_date]

oversampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Créez un modèle Random Forest avec les meilleurs paramètres trouvés
rf_classifier = RandomForestClassifier(
    n_estimators=100, 
    max_features=10, 
    max_depth=10, 
    criterion='entropy',
    random_state=1
)

# Entraînez le modèle sur l'ensemble d'entraînement équilibré
rf_classifier.fit(X_train_resampled, y_train_resampled)

# Faites des prédictions sur l'ensemble de test
y_pred = rf_classifier.predict(X_test)

# Évaluez le modèle
accuracy_selected = accuracy_score(y_test, y_pred)
classification_report_result_selected = classification_report(y_test, y_pred)

# Affichez les résultats
print(f"\nAccuracy with Selected Features: {accuracy_selected:.4f}")
print("Classification Report with Selected Features:\n", classification_report_result_selected)

KeyError: "['Points_Cum_Lag1', 'GD_Cum_Lag1', 'GF_Cum_Lag1', 'GA_Cum_Lag1'] not in index"